In [1]:
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.data import Data, DataLoader
import pandas as pd
import numpy as np
import h5py
from torch.cuda.amp import autocast, GradScaler

import dgl
import os
from dgl.data import DGLDataset
from def_load_data import get_model_data
from TC_EGAT import TTST_module

device = (
    torch.device("cuda")
    if torch.cuda.is_available()
    else torch.device("cpu")
)
device = torch.device("cpu")
ss_x,ss_y,x_train,y_train,x_val,y_val,x_test,y_test,nor_sta_fes,nor_edge_fes,G = get_model_data(device)

/home/jovyan/anaconda3/envs/cgnn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


20170416
(173, 23140, 8)
(23140, 8)
(144, 8, 30, 23140)
(144, 23140)
12
torch.Size([108, 8, 30, 23140]) torch.Size([12, 8, 30, 23140]) torch.Size([24, 8, 30, 23140])
torch.Size([108, 23140]) torch.Size([12, 23140]) torch.Size([24, 23140])
torch.Size([23140, 12])


In [2]:
from def_load_data import generate_dt_points
date_list = generate_dt_points('20160409','20221210')
y_dt = date_list[31:]
train_dt = y_dt[:130]
val_dt = y_dt[130:130+14]
test_dt = y_dt[130+14:130+14+29]
print(len(test_dt))
#test_dt  #130,13,30=173
#print(test_dt)
lack_dt_train = ['D20171001','D20171013','D20190418','D20190430','D20191003','D20191015','D20201126','D20201208','D20210302', 'D20210314']
lack_dt_test = ['D20220520','D20220601','D20220625','D20220707','D20220719','D20220731','D20220812','D20220905','D20220917','D20221222']
lack_dt_idx_train = []
lack_dt_idx_test = []
for i,dtc in enumerate(train_dt):
    for l_dt in lack_dt_train:
        if l_dt[1:] ==str(dtc):
            lack_dt_idx_train.append(i)
for i,dtc in enumerate(test_dt):
    for l_dt in lack_dt_test:
        if l_dt[1:] ==str(dtc):
            lack_dt_idx_test.append(i)
            
print(len(lack_dt_idx_train),len(lack_dt_idx_test))


lack_dt_idx_train = torch.tensor(lack_dt_idx_train)
lack_dt_idx_test = torch.tensor(lack_dt_idx_test)

train_keep = torch.tensor([i for i in range(x_train.size(0)) if i not in lack_dt_idx_train]).to(device)
test_keep = torch.tensor([i for i in range(x_test.size(0)) if i not in lack_dt_idx_test]).to(device)

# 删除指定索引的数据
x_train_d = torch.index_select(x_train, 0, train_keep)
y_train_d = torch.index_select(y_train, 0, train_keep)

x_test_d = torch.index_select(x_test, 0, test_keep)
y_test_d = torch.index_select(y_test, 0, test_keep)

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(x_train_d.shape)

29
10 9
torch.Size([108, 8, 30, 23140]) torch.Size([108, 23140])
torch.Size([24, 8, 30, 23140]) torch.Size([24, 23140])
torch.Size([102, 8, 30, 23140])


In [3]:
from load_data import generate_dt_points
date_list = generate_dt_points('20160409','20221210')
y_dt = date_list[31+30:]
train_dt = y_dt[:108]
val_dt = y_dt[108:108+12]
test_dt = y_dt[108+12:108+12+24]
print(len(test_dt))

lack_dt_train = ['D20171001','D20171013','D20190418','D20190430','D20191003','D20191015','D20201126','D20201208','D20210302', 'D20210314']
lack_dt_test = ['D20220520','D20220601','D20220625','D20220707','D20220719','D20220731','D20220812','D20220905','D20220917']
lack_dt_idx_train = []
lack_dt_idx_test = []
for i,dtc in enumerate(train_dt):
    for l_dt in lack_dt_train:
        if l_dt[1:] ==str(dtc):
            lack_dt_idx_train.append(i)
for i,dtc in enumerate(test_dt):
    for l_dt in lack_dt_test:
        if l_dt[1:] ==str(dtc):
            lack_dt_idx_test.append(i)
            
print(len(lack_dt_idx_train),len(lack_dt_idx_test))

lack_dt_idx_train = torch.tensor(lack_dt_idx_train)
lack_dt_idx_test = torch.tensor(lack_dt_idx_test)

train_keep = torch.tensor([i for i in range(x_train.size(0)) if i not in lack_dt_idx_train]).to(device)
test_keep = torch.tensor([i for i in range(x_test.size(0)) if i not in lack_dt_idx_test]).to(device)

# 删除指定索引的数据
x_train_d = torch.index_select(x_train, 0, train_keep)
y_train_d = torch.index_select(y_train, 0, train_keep)

x_test_d = torch.index_select(x_test, 0, test_keep)
y_test_d = torch.index_select(y_test, 0, test_keep)

print(x_train.shape,y_train.shape)
print(x_test_d.shape,y_test_d.shape)
print(x_train_d.shape)

23
8 9
torch.Size([108, 8, 30, 23140]) torch.Size([108, 23140])
torch.Size([15, 8, 30, 23140]) torch.Size([15, 23140])
torch.Size([100, 8, 30, 23140])


In [27]:
batch_size = 1
train_data = torch.utils.data.TensorDataset(x_train_d, y_train_d)
train_iter = torch.utils.data.DataLoader(train_data, batch_size)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)
test_data = torch.utils.data.TensorDataset(x_test_d, y_test_d)
test_iter = torch.utils.data.DataLoader(test_data, batch_size)

#del x_train,y_train,x_val,y_val,x_test,y_test

In [4]:
x_test_d.shape

torch.Size([21, 11, 30, 23140])

In [17]:
batch_size = 1
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)
test_data = torch.utils.data.TensorDataset(x_test, y_test)
test_iter = torch.utils.data.DataLoader(test_data, batch_size)

#del x_train,y_train,x_val,y_val,x_test,y_test

In [6]:
gpu_count = torch.cuda.device_count()
print("GPU 数量:", gpu_count)

GPU 数量: 2


In [28]:
c_in, c_out,sta_fes, sta_h1 = 8,32,12,12
droput = 0.5
device = torch.device("cuda:1")
tcg = TTST_module(c_in, c_out,sta_fes, sta_h1,droput).to(device)

total_params = 0
for name, param in tcg.named_parameters():
    if param.requires_grad:
        param_params = param.numel()
        total_params += param_params
        print(f"Layer: {name}, Parameters: {param_params}")

#print(f"Total Parameters: {total_params}")

Layer: T1.convT.bias, Parameters: 32
Layer: T1.convT.weight_g, Parameters: 32
Layer: T1.convT.weight_v, Parameters: 768
Layer: T1.conv.weight, Parameters: 768
Layer: T1.conv.bias, Parameters: 32
Layer: S1.fc1.weight, Parameters: 144
Layer: S1.fc1.bias, Parameters: 12
Layer: S1.convS.weight, Parameters: 144
Layer: S1.convS.bias, Parameters: 12
Layer: T2.convT.bias, Parameters: 64
Layer: T2.convT.weight_g, Parameters: 64
Layer: T2.convT.weight_v, Parameters: 8448
Layer: T2.conv.weight, Parameters: 8448
Layer: T2.conv.bias, Parameters: 64
Layer: S2.bottleneck.node.weight, Parameters: 3072
Layer: S2.bottleneck.node.bias, Parameters: 48
Layer: S2.bottleneck.edge.weight, Parameters: 64
Layer: S2.bottleneck.edge.bias, Parameters: 16
Layer: S2.egat.a_h_node, Parameters: 120
Layer: S2.egat.a_e_node, Parameters: 24
Layer: S2.egat.a_h_edge, Parameters: 120
Layer: S2.egat.a_e_edge, Parameters: 24
Layer: S2.egat.bias1, Parameters: 48
Layer: S2.egat.bias2, Parameters: 16
Layer: S2.egat.fh_n.weight, 

In [29]:
loss = nn.MSELoss()
lr =0.0008
optimizer = torch.optim.Adam(tcg.parameters(), lr=lr,weight_decay = 5e-5)  #RMSprop

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.8)

save_path = "TC_EGAT.pth"
min_val_loss = np.inf
scaler = GradScaler()

G = G.to(device)
nor_sta_fes = nor_sta_fes.to(device)
nor_edge_fes = nor_edge_fes.to(device)
#x_val,y_val = x_val.to(device),y_val.to(device)

torch.cuda.empty_cache()
for epoch in range(1, 200 + 1):
    l_sum, n = 0.0, 0
    tcg.train()
    for x, y in train_iter:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        with autocast():
            y_pred = tcg(G, x, nor_sta_fes,nor_edge_fes)
            l = loss(y_pred, y)
        #l.backward()
        #optimizer.step()
        scaler.scale(l).backward()
        scaler.step(optimizer)
        scaler.update()
        
        l_sum += l.item() * y.shape[0]
        n += y.shape[0]
    scheduler.step()

    val_loss = 0
    '''
    for x, y in val_iter:
        x,y= x.to(device),y.to(device)
        with autocast():
            val_pre = tcg(G, x, nor_sta_fes,nor_edge_fes)
        val_loss1 = loss(val_pre,y).item()
        val_loss += val_loss1* y.shape[0]
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(tcg.state_dict(), save_path)
    '''

    print("epoch",epoch,", train loss:",l_sum / n)

epoch 1 , train loss: 0.06708807537332177
epoch 2 , train loss: 0.058506665471941235
epoch 3 , train loss: 0.04014071312732995
epoch 4 , train loss: 0.03153025611303747
epoch 5 , train loss: 0.036073939362540844
epoch 6 , train loss: 0.0331950337998569
epoch 7 , train loss: 0.03045784207060933
epoch 8 , train loss: 0.02770428204908967
epoch 9 , train loss: 0.028000535517930983
epoch 10 , train loss: 0.02794991442002356
epoch 11 , train loss: 0.02638637657277286
epoch 12 , train loss: 0.025904695512726904
epoch 13 , train loss: 0.025873503303155302
epoch 14 , train loss: 0.026332028750330208
epoch 15 , train loss: 0.026918902099132537
epoch 16 , train loss: 0.025186858074739575
epoch 17 , train loss: 0.024340509325265883
epoch 18 , train loss: 0.023840955309569836
epoch 19 , train loss: 0.023627537665888668
epoch 20 , train loss: 0.023315162928774953
epoch 21 , train loss: 0.023000294882804154
epoch 22 , train loss: 0.02273289976641536
epoch 23 , train loss: 0.022483134707435966
epoch 2